# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [61]:
!cat publications_v2.tsv

pub_date	title	venue	citation	url_slug	paper_url	code_link
2021-05-21	2021_TPAMI_DSRL	IEEE Transactions on Pattern Analysis and Machine Intelligence	Shiping Wang, Zhaoliang Chen, Shide Du, and Zhouchen Lin. Learning deep sparse regularizers with applications to multi-view clustering and semi-supervised classification[J]. IEEE Transactions on Pattern Analysis and Machine Intelligence, 2021, 44(9): 5042-5055.	DSRL	https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9439159	https://github.com/chenzl23/DSRL
2023-07-01	2023_INF_LGCN-FF	Information Fusion	"Zhaoliang Chen, Lele Fu, Jie Yao, Wenzhong Guo, Claudia Plant, and Shiping Wang.
Learnable graph convolutional network and feature fusion for multi-view learning,
Information Fusion, 2023, 95: 109-119."	LGCN-FF	https://www.sciencedirect.com/science/article/pii/S1566253523000532	https://github.com/chenzl23/LGCNFF
2023-03-23	2023_TMM_IMvGCN	IEEE Transactions on Multimedia	Zhihao Wu, Xincan Lin, Zhenghong Lin, Zhaoliang Chen, Yang Bai,  

## Import pandas

We are using the very handy pandas library for dataframes.

In [62]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [63]:
publications = pd.read_csv("publications_v2.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,citation,url_slug,paper_url,code_link
0,2021-05-21,2021_TPAMI_DSRL,IEEE Transactions on Pattern Analysis and Mach...,"Shiping Wang, Zhaoliang Chen, Shide Du, and Zh...",DSRL,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...,https://github.com/chenzl23/DSRL
1,2023-07-01,2023_INF_LGCN-FF,Information Fusion,"Zhaoliang Chen, Lele Fu, Jie Yao, Wenzhong Guo...",LGCN-FF,https://www.sciencedirect.com/science/article/...,https://github.com/chenzl23/LGCNFF
2,2023-03-23,2023_TMM_IMvGCN,IEEE Transactions on Multimedia,"Zhihao Wu, Xincan Lin, Zhenghong Lin, Zhaolian...",IMvGCN,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...,https://github.com/ZhihaoWu99/IMvGCN
3,2023-01-11,2023_AAAI_tsGCN,AAAI 2023,"Shiping Wang, Zhihao Wu, Yuhong Chen, and Yong...",tsGCN,https://scholar.google.com/citations?view_op=v...,https://github.com/ZhihaoWu99/tsGCN
4,2023-06-26,2023_AAAI_DLR-GAE,AAAI 2023,"Zhaoliang Chen, Zhihao Wu, Shiping Wang, and W...",DLR-GAE,https://ojs.aaai.org/index.php/AAAI/article/do...,https://github.com/chenzl23/DLRGAE
5,2022-07-01,2022_TIP_MSA-Net,IEEE Transactions on Multimedia,"Linxin Zheng, Guobao Xiao, Ziwei Shi, Shiping ...",MSA-Net,https://guobaoxiao.github.io/papers/TIP2022b(1...,https://github.com/guobaoxiao/MSANet
6,2022-06-24,2022_TMM_ULTLSE,IEEE Transactions on Multimedia,"Lele Fu, Zhaoliang Chen, Yongyong Chen, and Sh...",ULTLSE,https://chenzl23.github.io/assets/pdf/ULTLSE-T...,NaN
7,2022-03-09,2022_TMM_LMMF,IEEE Transactions on Multimedia,"Sheng Huang, Yunhe Zhang, Lele Fu, and Shiping...",LMMF,https://ieeexplore.ieee.org/stamp/stamp.jsp?tp...,NaN
8,2022-03-01,2022_PR_DCCA,Pattern Recognition,"Jinyu Cai, Shiping Wang, Chaoyang Xu, and Wenz...",DCCA,https://www.sciencedirect.com/science/article/...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [64]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [65]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    # if len(str(item.excerpt)) > 5:
    #     md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    if len(str(item.code_link)) > 5:
        md += "\ncodelink: '" + item.code_link + "'"

    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\nurlslug: '" + html_escape(item.url_slug) + "'"

    md += "\n---"
    
    ## Markdown description for individual page
        
    # if len(str(item.excerpt)) > 5:
    #     md += "\n" + html_escape(item.excerpt) + "\n"
    
    # if len(str(item.paper_url)) > 5:
    #     md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [66]:
!ls ../_publications/

2021-05-21-DSRL.md
2022-03-01-DCCA.md
2022-03-09-LMMF.md
2022-06-24-ULTLSE.md
2022-07-01-MSA-Net.md
2023-01-11-tsGCN.md
2023-03-23-IMvGCN.md
2023-06-26-DLR-GAE.md
2023-07-01-LGCN-FF.md


In [67]:
!cat ../_publications/2021-05-21-DSRL.md

---
title: "2021_TPAMI_DSRL"
collection: publications
permalink: /publication/2021-05-21-DSRL
date: 2021-05-21
venue: 'IEEE Transactions on Pattern Analysis and Machine Intelligence'
paperurl: 'https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9439159'
codelink: 'https://github.com/chenzl23/DSRL'
citation: 'Shiping Wang, Zhaoliang Chen, Shide Du, and Zhouchen Lin. Learning deep sparse regularizers with applications to multi-view clustering and semi-supervised classification[J]. IEEE Transactions on Pattern Analysis and Machine Intelligence, 2021, 44(9): 5042-5055.'
urlslug: 'DSRL'
---
Recommended citation: Shiping Wang, Zhaoliang Chen, Shide Du, and Zhouchen Lin. Learning deep sparse regularizers with applications to multi-view clustering and semi-supervised classification[J]. IEEE Transactions on Pattern Analysis and Machine Intelligence, 2021, 44(9): 5042-5055.
